In [6]:
# Importar bibliotecas necesarias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import resample

# Cargar dataset desde GitHub
url = "https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv"
data = pd.read_csv(url)

# Renombrar columnas para adaptarlas al script
data = data[['tweet', 'label']]
data.rename(columns={'tweet': 'text', 'label': 'category'}, inplace=True)

# Mapear etiquetas numéricas a texto (opcional)
data['category'] = data['category'].map({0: 'negativo', 1: 'positivo'})

# Previsualizar datos
print(data.head())

# Rebalancear las clases con sobremuestreo
data_balanced = pd.concat([data['text'], data['category']], axis=1)

# Separar clases
negativo = data_balanced[data_balanced['category'] == 'negativo']
positivo = data_balanced[data_balanced['category'] == 'positivo']

# Sobremuestrear la clase positiva
positivo_upsampled = resample(positivo,
                              replace=True,     # Sample with replacement
                              n_samples=len(negativo), # Igualar al número de negativos
                              random_state=42)

# Combinar las clases
data_balanced = pd.concat([negativo, positivo_upsampled])

# Mezclar los datos
data_balanced = data_balanced.sample(frac=1, random_state=42)

# Separar X e y
X = data_balanced['text']
y = data_balanced['category']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizar texto con TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Entrenar modelo de regresión logística con pesos equilibrados
model = LogisticRegression(class_weight='balanced', random_state=42)
model.fit(X_train_tfidf, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Función para clasificar texto nuevo
def clasificar_texto(nuevo_texto):
    nuevo_texto_tfidf = vectorizer.transform([nuevo_texto])
    prediccion = model.predict(nuevo_texto_tfidf)[0]
    return prediccion

# Probar la función con un ejemplo
ejemplo = "The product was amazing and worked perfectly!"
print(f"Texto: '{ejemplo}'")
print(f"Clasificación: {clasificar_texto(ejemplo)}")

# Ajustar el umbral para mejorar la clasificación de textos positivos
y_probs = model.predict_proba(X_test_tfidf)

# Ajustar el umbral
nuevo_umbral = 0.4
y_pred_adjusted = ['positivo' if prob[1] > nuevo_umbral else 'negativo' for prob in y_probs]

# Evaluar el modelo ajustado
print("Accuracy (con umbral ajustado):", accuracy_score(y_test, y_pred_adjusted))
print("Classification Report (con umbral ajustado):\n", classification_report(y_test, y_pred_adjusted))


                                                text  category
0   @user when a father is dysfunctional and is s...  negativo
1  @user @user thanks for #lyft credit i can't us...  negativo
2                                bihday your majesty  negativo
3  #model   i love u take with u all the time in ...  negativo
4             factsguide: society now    #motivation  negativo
Accuracy: 0.958193135935397
Classification Report:
               precision    recall  f1-score   support

    negativo       0.98      0.93      0.96      5899
    positivo       0.94      0.98      0.96      5989

    accuracy                           0.96     11888
   macro avg       0.96      0.96      0.96     11888
weighted avg       0.96      0.96      0.96     11888

Texto: 'The product was amazing and worked perfectly!'
Clasificación: negativo
Accuracy (con umbral ajustado): 0.943893001345895
Classification Report (con umbral ajustado):
               precision    recall  f1-score   support

    negativo 